<a href="https://colab.research.google.com/github/Chen-Mei-Chen/Python_Self_Learning/blob/main/%3CPython%E7%AD%86%E8%A8%98%3E%E6%AA%94%E6%A1%88%E8%B7%AF%E5%BE%91(Path%E7%B7%B4%E7%BF%92).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

之前有提過利用os方式進行檔案管理，**os主要是以字串的形式定義路徑，而Path則是物件導向**，現代框架幾乎都用Path。

以下是四個(Chatgpt提供的)小練習，實作後就大致會使用了:

* 練習一：建立與檢查路徑
* 練習二：列出資料夾內容並過濾
* 練習三：取得檔案資訊（名稱、副檔名、父資料夾）
* 練習四：用 Path 取代 os.path 的程式片段
* 加分活動一：使用 glob() 遞迴找圖檔
* 加分活動二：搬移指定副檔名的檔案

(這篇notebook是在kaggle寫成，所以路徑部分使用不少kaggle的路徑型態)

## **練習一：建立與檢查路徑:**

假設你要在當前目錄下建立以下結構：

請用 pathlib 建立這個資料夾結構（如果已存在就跳過）。

然後檢查 images 資料夾是否存在並且是資料夾。

In [ ]:
project/
└── data/
    └── images/

In [ ]:
#解答:

from pathlib import Path

p = Path("/kaggle/working/project/data/images") #記得新定義路徑要括號+雙引號!!
p.mkdir(parents=True, exist_ok=True)

if p.exists() and p.is_dir():
    print("資料夾存在")
else:
    print("路徑不存在或不是資料夾")

資料夾存在


## **練習二:列出資料夾內容並過濾**

假設你已經有一個資料夾 project/data/images/，裡面有許多檔案與子資料夾（有 .jpg, .png, .txt, 其他子資料夾）。
* 請使用 iterdir() 列出所有子項目。
* 過濾只印出 .jpg 和 .png 副檔名的檔案。

（可用屬性 .suffix 來判斷）

In [ ]:
#解答:

#首先我們要先建立各種不同的空檔案，既然母路徑是一樣的，有一個聰明的方式:遞迴:
base = Path("/kaggle/working/project/data/images")

for name in ["cat.jpg", "dog.png", "note.txt", "readme.md"]:
    (base / name).touch()  # .touch():建立空檔案  然後記得新定義路徑要括號+雙引號!!

#子資料夾:
(base / "dir2").mkdir(exist_ok=True)

#印出圖片檔:利用尋找檔案好幫手:iterdir、分類檔名好幫手suffix
for item in base.iterdir():
    if item.suffix in ['.jpg', '.png']:
        print("圖片檔：", item.name)

圖片檔： dog.png
圖片檔： cat.jpg


## **練習三：取得檔案資訊（名稱、副檔名、父資料夾)**

請你自行指定一個檔案路徑，例如 "project/data/images/cat.jpg"，然後用 Path 方法印出：
* 取得檔案名稱：cat.jpg
* 取得檔名（不含副檔名）：cat
* 副檔名：.jpg
* 上層資料夾名稱：應該是 images

In [ ]:
#解答:

#檔名:
p = Path("/kaggle/working/project/data/images/cat.jpg")
print(f'取得檔案名稱：{base.name}')

#下面容我偷懶，不寫成format:
print(p.stem)
print(p.suffix)  #真眼熟
print(p.parent.name)

取得檔案名稱：images
cat
.jpg
images


## **練習四：用 Path 取代 os.path 的程式片段**

下面是一段使用 os 的程式邏輯，請你改寫成使用 pathlib.Path 的版本：

In [ ]:
#題目:

import os

folder = "project/data/images"
filename = "dog.png"
filepath = os.path.join(folder, filename)

if os.path.exists(filepath) and os.path.isfile(filepath):
    size = os.path.getsize(filepath)
    print("檔案大小:", size)
else:
    print("檔案不存在或非檔案")

In [ ]:
#解答:

path = Path("/kaggle/working/project/data/images/dog.png")

if path.exists() and path.is_file():
    print(f"檔案大小:{path.stat().st_size}") #這就是檔案大小的求法
else:
    print("檔案不存在或非檔案")

檔案大小:0


## **加分活動一：使用 glob() 遞迴找圖檔**

請你搜尋 project/data/images/ 資料夾下所有子目錄裡的 .jpg 圖片，並列出它們的完整路徑。

**ps.常用的 glob() 通配符總整理:**

| 通配符      | 意義/匹配內容                    | 範例               | 匹配範例                                        |
| -------- | -------------------------- | ---------------- | ------------------------------------------- |
| `*`      | 任意長度的字元（不含 `/`）            | `*.jpg`          | `a.jpg`, `photo.jpg`, `123.jpg`             |
| `?`      | 任意一個字元                     | `file?.txt`      | `file1.txt`, `fileA.txt`（不含 `file12.txt`）   |
| `[abc]`  | 任一個 `a`、`b`、或 `c`          | `file[ab].txt`   | `filea.txt`, `fileb.txt`                    |
| `[a-z]`  | 範圍內任一字元（a 到 z）             | `file[a-z].txt`  | `filem.txt`, `filez.txt`                    |
| `[!abc]` | **不是** `a`、`b`、或 `c` 的任一字元 | `file[!ab].txt`  | `filex.txt`, `file9.txt`（不含 `filea.txt`）    |
| `[!a-z]` | **不是** 小寫英文字的任一字元          | `file[!a-z].txt` | `file1.txt`, `fileA.txt`（不含 `filea.txt`）    |
| `**`     | 遞迴所有子資料夾（含多層）              | `**/*.jpg`       | `img.jpg`, `folder1/cat.jpg`, `a/b/c/d.jpg` |

In [ ]:
#解答:

p = Path("/kaggle/working/project/data/images")

for file in p.glob("**/*.jpg"):
    print(file.resolve())  # 用resolve()顯示絕對路徑

#glob就像探員，resolve就像記者

/kaggle/working/project/data/images/cat.jpg


## **加分活動二：搬移指定副檔名的檔案**

將 project/data/images/ 下所有 .png 圖片（包含子資料夾）複製到 project/data/pngs/ 資料夾，若目的資料夾不存在，請自動建立。

**ps.複製路徑shutil.copy(file, dst_dir / file.name)的說明:**

shutil.copy(複製檔案的完整路徑，可以是Path也可以是字串, 目的檔案的完整路徑 / 複製檔案.name)

所以輸出的新檔案就是:目的檔路徑/複製檔

In [ ]:
#解答:

import shutil

path = Path("/kaggle/working/project/data/pngs")
path.mkdir(parents=True, exist_ok=True)

for file in p.glob("**/*.png"):
    shutil.copy(file, path / file.name)